# Exercise 2: Spark Fundamentals

## Learning Objectives
- Understand the difference between RDDs and DataFrames
- Master transformations (lazy) vs actions (eager)
- Create and manipulate distributed datasets
- View job execution in the Spark UI

---

## Part 1: Creating a SparkSession

In [1]:
from pyspark.sql import SparkSession

# Create SparkSession connected to YARN
spark = SparkSession.builder \
    .appName("Spark Fundamentals Lab") \
    .master("yarn") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Application ID: {spark.sparkContext.applicationId}")

Spark Version: 3.5.0
Application ID: application_1768525703064_0001


🔗 **Open Spark UI**: http://localhost:4040 to monitor your application

## Part 2: Working with RDDs (Resilient Distributed Datasets)

RDDs are the foundational data structure in Spark - immutable, distributed collections.

In [20]:
# Create an RDD from a Python list
numbers = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], numSlices=2)
print(f"Number of partitions: {numbers.getNumPartitions()}")

Number of partitions: 2


In [6]:
# Transformations are LAZY - they don't execute immediately
squared = numbers.map(lambda x: x ** 2)
filtered = squared.filter(lambda x: x > 20)

print("Transformations defined, but nothing executed yet!")
print(f"Type: {type(filtered)}")

Transformations defined, but nothing executed yet!
Type: <class 'pyspark.rdd.PipelinedRDD'>


In [7]:
# Actions TRIGGER execution
result = filtered.collect()
print(f"Result: {result}")

Result: [25, 36, 49, 64, 81, 100]


### 🔍 Checkpoint Question 2
Check the Spark UI (Jobs tab). How many jobs were triggered by the code above?
Which operation triggered the job - `map()`, `filter()`, or `collect()`?

**Your Answer:**

## Part 3: Common RDD Transformations and Actions

In [9]:
# Read file as RDD
lines_rdd = spark.sparkContext.textFile("hdfs:///user/student/data/transactions.csv")

# Skip header and parse
data_rdd = lines_rdd.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])

In [10]:
# Transformations
def parse_transaction(line):
    fields = line.split(',')
    return {
        'transaction_id': fields[0],
        'amount': float(fields[7]) if len(fields) > 7 else 0.0
    }

parsed = data_rdd.map(parse_transaction)
amounts = parsed.map(lambda x: x['amount'])

In [11]:
# Actions
print(f"Count: {amounts.count()}")
print(f"Sum: ${amounts.sum():,.2f}")
print(f"Average: ${amounts.mean():,.2f}")
print(f"Max: ${amounts.max():,.2f}")

Count: 100000
Sum: $130,918,536.52
Average: $1,309.19
Max: $4,990.80


## Part 4: DataFrames - The Modern API

DataFrames are structured, schema-aware, and optimized with Catalyst.

In [12]:
# Read CSV with schema inference
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs:///user/student/data/transactions.csv")

df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_time: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- store_region: string (nullable = true)
 |-- is_online: boolean (nullable = true)



In [13]:
# Show sample data
df.show(5)

+--------------+----------------+-------------------+-----------+----------+--------+----------+------------+--------------+------------+---------+
|transaction_id|transaction_date|   transaction_time|customer_id|product_id|quantity|unit_price|total_amount|payment_method|store_region|is_online|
+--------------+----------------+-------------------+-----------+----------+--------+----------+------------+--------------+------------+---------+
|   TXN00000001|      2024-06-03|2026-01-11 11:56:54| CUST000481| PROD00352|       1|    259.06|      259.06|        PayPal|       North|     true|
|   TXN00000002|      2023-11-22|2026-01-11 14:25:51| CUST001581| PROD00288|       6|     442.7|      2656.2|        PayPal|     Central|    false|
|   TXN00000003|      2023-08-13|2026-01-11 13:40:52| CUST000534| PROD00205|       1|     116.0|       116.0|   Credit Card|     Central|    false|
|   TXN00000004|      2023-06-20|2026-01-11 16:13:49| CUST000840| PROD00230|       4|    187.53|      750.12|   

In [14]:
# DataFrame transformations
from pyspark.sql.functions import col, sum, avg, count

# Group by region and calculate totals
region_summary = df.groupBy("store_region") \
    .agg(
        count("*").alias("transaction_count"),
        sum("total_amount").alias("total_sales"),
        avg("total_amount").alias("avg_transaction")
    ) \
    .orderBy("total_sales", ascending=False)

region_summary.show()

+------------+-----------------+--------------------+------------------+
|store_region|transaction_count|         total_sales|   avg_transaction|
+------------+-----------------+--------------------+------------------+
|        West|            21575|2.8312219759999976E7| 1312.269745538817|
|       North|            20048| 2.652123458999993E7|1322.8868011771713|
|       South|            20191| 2.649756461000003E7|1312.3453325739206|
|     Central|            19364|2.5204887519999914E7|1301.6364139640525|
|        East|            18822|2.4382630039999966E7|1295.4324747635726|
+------------+-----------------+--------------------+------------------+



## Part 5: Understanding Lazy Evaluation

In [15]:
# These are all transformations - nothing executes yet!
step1 = df.filter(col("is_online") == True)
step2 = step1.select("transaction_id", "total_amount", "payment_method")
step3 = step2.filter(col("total_amount") > 100)

print("Built the execution plan. Check Spark UI - no new jobs yet!")

Built the execution plan. Check Spark UI - no new jobs yet!


In [16]:
# View the execution plan
step3.explain(True)

== Parsed Logical Plan ==
'Filter ('total_amount > 100)
+- Project [transaction_id#17, total_amount#24, payment_method#25]
   +- Filter (is_online#27 = true)
      +- Relation [transaction_id#17,transaction_date#18,transaction_time#19,customer_id#20,product_id#21,quantity#22,unit_price#23,total_amount#24,payment_method#25,store_region#26,is_online#27] csv

== Analyzed Logical Plan ==
transaction_id: string, total_amount: double, payment_method: string
Filter (total_amount#24 > cast(100 as double))
+- Project [transaction_id#17, total_amount#24, payment_method#25]
   +- Filter (is_online#27 = true)
      +- Relation [transaction_id#17,transaction_date#18,transaction_time#19,customer_id#20,product_id#21,quantity#22,unit_price#23,total_amount#24,payment_method#25,store_region#26,is_online#27] csv

== Optimized Logical Plan ==
Project [transaction_id#17, total_amount#24, payment_method#25]
+- Filter ((isnotnull(is_online#27) AND isnotnull(total_amount#24)) AND (is_online#27 AND (total_amou

In [17]:
# Trigger execution with an action
result_count = step3.count()
print(f"High-value online transactions: {result_count}")

High-value online transactions: 47192


## Cleanup

In [18]:
# Stop SparkSession when done
spark.stop()
print("SparkSession stopped.")

SparkSession stopped.
